In [41]:
import numpy as np
import matplotlib.pyplot as plt

In [42]:
class Plane:
    def __init__(self,_id,_current_pos,_current_vel,_pos_array,_vel_array):
        self.id = _id #Unique id of plane
        #Think remove the initial pos
        self.pos_current = _current_pos #Pos at current time
        self.vel_current = _current_vel #Vel at current time
        self.pos_array = _pos_array #Array of position over time
        self.vel_array = _vel_array #Array of velocity over time

    def __str__(self):
        #For useful output whenprinting object e.g. print(plane) = ID: 10, Pos: [0,1,2] etc.
        #Probably needs bit of adjusting
        return "ID: {id}, Current_pos: {c_pos}, Current_vel: {c_vel}".format(id = self.id, c_pos = self.pos_current, c_vel = self.vel_current)

In [43]:
def create_n_planes(n_planes,max_pos,max_vel,num_time_steps):
    # n_planes controls how many planes are created
    # max_pos is used as size of the grid planes put randomly in (just temp)
    # max_vel scales the maximum velocity of the planes

    plane_list = []
    for id in range(n_planes):
        # initial_pos = np.random.rand(3) * max_pos
        # initial_vel = np.random.rand(3) * max_vel

        pos_array = np.zeros((num_time_steps,3))
        vel_array = np.zeros((num_time_steps,3))

        pos_array[0,:] = np.random.rand(3) * max_pos
        vel_array[0,:] = np.random.rand(3) * max_vel

        plane_list.append(Plane(id, #Unique id
                                pos_array[0,:], #Inital pos
                                vel_array[0,:], #Initial vel
                                pos_array, #Pos array
                                vel_array)) #Vel array
    
    return plane_list

In [44]:
def calc_distances(planes,time_step):
    #Returns n x n matrix of distance between plane i and plane j
    #Diagonals should be zero
    #Should be symmetrical
    num_planes = len(planes)
    out = np.zeros((num_planes,num_planes))

    for i in range(num_planes):
        for j in range(i+1,num_planes):
            pos_i = planes[i].pos_array[time_step,:]
            pos_j = planes[j].pos_array[time_step,:]
            distance = np.linalg.norm(pos_i - pos_j)
            out[i,j] = distance

    return out

In [45]:
#Time setup for solving
max_t = 30
delta_t = 0.5
num_t_steps = int(max_t / delta_t)

#Square grid size
grid_size = 100

#Define variables about planes
num_planes = 10
max_plane_vel = 10

#Uncertainty magnitude
uncertainty = max_plane_vel/8

#Iteration variables
iteration = 0
iteration_max = 1000
#Separation threshold, below which = crash
separation_threshold = 5

In [46]:
#Output
total_crashed = 0

while iteration < iteration_max:
    #Create plane objects
    planes = create_n_planes(num_planes,grid_size,max_plane_vel,num_t_steps)

    #Create variables to store distances
    distances = np.zeros((num_t_steps-1,num_planes,num_planes))
    #avg_distances = np.zeros((num_t_steps-1))
    crashed = np.zeros((num_planes,num_planes))

    for timestep in range(num_t_steps-1):
        distances[timestep,:,:] = calc_distances(planes,timestep)
        #avg_distances[timestep] = np.mean(distances[timestep,:,:])
        crashed[np.where((distances[timestep,:,:]<separation_threshold)&(distances[timestep,:,:]!=0),1,0).nonzero()] = 1
        for plane in planes:
            #Extract pos and vel
            pos = plane.pos_array[timestep,:]
            vel = plane.vel_array[timestep,:]

            #Increment pos and vel
            new_pos = pos + vel * delta_t
            new_vel = vel + np.random.uniform(-1,1,3) * uncertainty

            #Store new val
            plane.pos_array[timestep+1,:] = new_pos
            plane.vel_array[timestep+1,:] = new_vel

    #Count total crashed
    total_crashed += np.sum(crashed)

    iteration += 1

print("Total crashes: {total_crashes}, Iterations: {iter_max}".format(total_crashes = total_crashed, iter_max = iteration_max))
print("Mean crashes per run: {mean_crash}".format(mean_crash = total_crashed / iteration_max))

Total crashes: 152.0, Iterations: 1000
Mean crashes per run: 0.152
